In [12]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate

from langchain.llms import CTransformers

In [13]:
# i need to create my pinecone cluster because i will be using pinecone vector db
PINECONE_API_KEY = "a9503f39-6c3e-4ba9-bf17-61a684410cc0"
PINECONE_API_ENV = "gcp-starter"

In [14]:
#Extract data from the pdf
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()
    return documents

In [15]:
extracted_data = load_pdf("data/")

In [16]:
# Now i am converting the data into chunks
# create text chunks

def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap= 20)
    # print(text_splitter)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [17]:
text_chunks = text_split(extracted_data)
print("Length of my chunks",len(text_chunks))

Length of my chunks 7020


In [18]:
# NOW i want to convert chunks to vector embedding

def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings
    

In [19]:
embeddings = download_hugging_face_embeddings()

In [20]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [21]:
query_result = embeddings.embed_query("Hello world")
print("Length",len(query_result))

Length 384


In [22]:
# query_result

In [23]:
#Initializing Pinecone
pinecone.init(api_key=PINECONE_API_KEY,
              environment=PINECONE_API_ENV)
index_name = "medical-chatbot"

#creating embedding for each of the text chunk & storing
#docsearch is nothing but knowledge base/Vector db representation.

docsearch = Pinecone.from_texts([t.page_content for t in text_chunks],embeddings,index_name=index_name)

In [24]:
docsearch = Pinecone.from_existing_index(index_name,embeddings)
query = "What are allergies"

docs = docsearch.similarity_search(query,k=3)
print("Result",docs)

Result [Document(page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain histamine.HistaminePollen grains\nLymphocyte\nFIRST EXPOSURE", metadata={}), Document(page_content='allergens are the following:\n• plant pollens\n• animal fur and dander\n• body parts from house mites (microscopic creatures\nfound in all houses)\n• house dust• mold spores• cigarette smoke• solvents• cleaners\nCommon food allergens include the following:\n• nuts, especially peanuts, walnuts, and brazil nuts\n• fish, mollusks, and shellfish• eggs• wheat• milk• food additives and preservatives\nThe following types of drug

In [25]:
prompt_template = """
Use the following piece of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context:{context}
Question:{question}

Only return the helpful answer and nothing else

Helpful answer:
"""


In [28]:
PROMPT = PromptTemplate(template=prompt_template,input_variables=["context","question"])
chain_type_kwargs = {"prompt":PROMPT}

In [29]:
llm = CTransformers(model = "model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                    model_type = "llama",
                    config={'max_new_tokens':512,
                            'temperature':0.8})


In [30]:
#Question/Answering Object
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever = docsearch.as_retriever(search_kwargs ={'k':2}),
    return_source_documents = True,
    chain_type_kwargs=chain_type_kwargs)

In [1]:
while True:
    user_input = input(f"Input Prompt:")
    result = qa({"query":user_input})
    print("Response",result["result"])